# Sistemas de Recomendación

## Carga de Librerias

In [1]:
import numpy as np
import pandas as pd
import scipy
import sys
import re
#nltk.download()
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# pip install unidecode
sys.path.append(r"Utilities")
import Utilities 

## Lectura DataSet de Juguetes (usar encoding='iso-8859-1')

In [2]:
dataset = pd.read_csv('DatosJuguetes/datasetJuguetes.csv',delimiter=";",encoding='iso-8859-1')
dataset

,Url,Titulo,Descripcion,Tipo,Idioma
0,http://www.toysrus.es/product/index.jsp?produc...,Asalvo - Silla de Paseo Elba Rosa,Comprar Asalvo - Silla de Paseo Elba Rosa - La...,product,es_ES
1,http://www.toysrus.es/product/index.jsp?produc...,Zobo - Silla Paseo Moving Light,Comprar Zobo - Silla Paseo Moving Light - Zobo...,product,es_ES
2,http://www.toysrus.es/product/index.jsp?produc...,Maclaren - Silla Paseo Quest Medieval Azul/Plata,Comprar Maclaren - Silla Paseo Quest Medieval ...,product,es_ES
3,http://www.toysrus.es/product/index.jsp?produc...,Chicco - Silla Paseo London - Red Passion,Comprar Chicco - Silla Paseo London - Red Pass...,product,es_ES
4,http://www.toysrus.es/product/index.jsp?produc...,Chicco - Silla Paseo London - Matrix,Comprar Chicco - Silla Paseo London - Matrix -...,product,es_ES
...,...,...,...,...,...
5592,http://www.toysrus.es/product/index.jsp?produc...,Magic Box Huella,"Comprar Magic Box Huella - Magic Box Huella, u...",product,es_ES
5593,http://www.toysrus.es/product/index.jsp?produc...,Bright Starts - Mordedores Nuby,Comprar Bright Starts - Mordedores Nuby - Los ...,product,es_ES
5594,http://www.toysrus.es/product/index.jsp?produc...,Bright Stats - Mordedor Libro (varios modelos),Comprar Bright Stats - Mordedor Libro (varios ...,product,es_ES
5595,http://www.toysrus.es/product/index.jsp?produc...,Peppa Pig - Toalla de Playa,Comprar Peppa Pig - Toalla de Playa - Peppa Pi...,product,es_ES


## Breve Análsis descriptivo

In [3]:
dataset.isnull().sum()

Url            0
Titulo         0
Descripcion    0
Tipo           0
Idioma         0
dtype: int64

##### Conclusiones
- Hay 5 columnas de tipo string
- Se tiene un dataset de 5597x5
- No hay missing values

## Listado de opciones para recomendar (Listado juguetes)

In [4]:
Listado = dataset['Titulo']
Listado

0                      Asalvo - Silla de Paseo Elba Rosa
1                        Zobo - Silla Paseo Moving Light
2       Maclaren - Silla Paseo Quest Medieval Azul/Plata
3              Chicco - Silla Paseo London - Red Passion
4                   Chicco - Silla Paseo London - Matrix
                              ...                       
5592                                    Magic Box Huella
5593                     Bright Starts - Mordedores Nuby
5594      Bright Stats - Mordedor Libro (varios modelos)
5595                         Peppa Pig - Toalla de Playa
5596                         Star Wars - Toalla de Playa
Name: Titulo, Length: 5597, dtype: object

## Cálculo vectores descriptivos de Items

##### Limpieza de datos

In [5]:
#Tokenization
dataset['Descripcion_tok'] = dataset['Descripcion'].apply(lambda x: Utilities.tokenizacion(x))
dataset['Descripcion_tok']

0       [comprar, asalvo, -, silla, paseo, elba, rosa,...
1       [comprar, zobo, -, silla, paseo, moving, light...
2       [comprar, maclaren, -, silla, paseo, quest, me...
3       [comprar, chicco, -, silla, paseo, london, -, ...
4       [comprar, chicco, -, silla, paseo, london, -, ...
                              ...                        
5592    [comprar, magic, box, huella, -, magic, box, h...
5593    [comprar, bright, starts, -, mordedores, nuby,...
5594    [comprar, bright, stats, -, mordedor, libro, (...
5595    [comprar, peppa, pig, -, toalla, playa, -, pep...
5596    [comprar, star, wars, -, toalla, playa, -, sta...
Name: Descripcion_tok, Length: 5597, dtype: object

In [6]:
# Eliminar palabras con ausencia de significaddo (remover stopwords)
dataset['Descripcion_tok'] = dataset['Descripcion_tok'].apply(lambda x:Utilities.removeStopwords(x))
dataset['Descripcion_tok'].head(10)                                                 

0    [asalvo, -, silla, paseo, elba, rosa, -, asalv...
1    [zobo, -, silla, paseo, moving, light, -, zobo...
2    [maclaren, -, silla, paseo, quest, medieval, a...
3    [chicco, -, silla, paseo, london, -, red, pass...
4    [chicco, -, silla, paseo, london, -, matrix, -...
5    [chicco, -, silla, paseo, echo, twin, -, garne...
6    [num, noms, -, pack, misterioso, (, varios, mo...
7    [princesas, disney, -, set, cenicienta, -, des...
8    [masks, -, figura, gatuno, (, catboy, ), luz, ...
9    [you, &, -, pack, baberos, toalla, -, you, &, ...
Name: Descripcion_tok, dtype: object

In [7]:
# Eliminar signos de puntuación (remover signos de puntuacion)
dataset['Descripcion_tok'] = dataset['Descripcion_tok'].apply(lambda x:Utilities.removePunctuation(x))
dataset['Descripcion_tok'].head(10) 

0    [asalvo, silla, paseo, elba, rosa, asalvo, sil...
1    [zobo, silla, paseo, moving, light, zobo, sill...
2    [maclaren, silla, paseo, quest, medieval, azul...
3    [chicco, silla, paseo, london, red, passion, c...
4    [chicco, silla, paseo, london, matrix, chicco,...
5    [chicco, silla, paseo, echo, twin, garnet, chi...
6    [num, noms, pack, misterioso, varios, modelos,...
7    [princesas, disney, set, cenicienta, descubre,...
8    [masks, figura, gatuno, catboy, luz, noche, mo...
9    [you, pack, baberos, toalla, you, pack, babero...
Name: Descripcion_tok, dtype: object

In [8]:
# Union descripciones (pasar de array a string)
dataset['Descripcion_clean'] = dataset['Descripcion_tok'].apply(lambda x:Utilities.arrayToString(x))
dataset['Descripcion_clean'].head(10)

0    asalvo silla paseo elba rosa asalvo silla pase...
1    zobo silla paseo moving light zobo silla paseo...
2    maclaren silla paseo quest medieval azul/plata...
3    chicco silla paseo london red passion chicco s...
4    chicco silla paseo london matrix chicco silla ...
5    chicco silla paseo echo twin garnet chicco sil...
6    num noms pack misterioso varios modelos num no...
7    princesas disney set cenicienta descubre set m...
8    masks figura gatuno catboy luz noche momento p...
9    you pack baberos toalla you pack baberos toall...
Name: Descripcion_clean, dtype: object

##### Cálculo Matriz Frecuencias (CountVectorizer(), usar encoding='iso-8859-1')

In [9]:
vectorizer = CountVectorizer(encoding='iso-8859-1')
MatrizFrecuencias = vectorizer.fit_transform(dataset['Descripcion_clean'])
MatrizFrecuencias

<5597x19147 sparse matrix of type '<class 'numpy.int64'>'
	with 208900 stored elements in Compressed Sparse Row format>

##### Visualización Matriz Frecuencias

In [10]:
X = pd.DataFrame(data=MatrizFrecuencias.toarray(), index=Listado.values,
                columns=vectorizer.get_feature_names())
X

C:\Users\rzambrano\Anaconda3\envs\general\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,_ardtaloqko,aaa,aaron,abajo,abalanzarse,abalorio,abalorios,abandonada,abandonadas,abandonar,...,única,únicamente,únicas,único,únicos,úsala,úsela,útil,útiles,überjackal
Asalvo - Silla de Paseo Elba Rosa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zobo - Silla Paseo Moving Light,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Maclaren - Silla Paseo Quest Medieval Azul/Plata,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Chicco - Silla Paseo London - Red Passion,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Chicco - Silla Paseo London - Matrix,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Magic Box Huella,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bright Starts - Mordedores Nuby,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bright Stats - Mordedor Libro (varios modelos),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Peppa Pig - Toalla de Playa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Porcentaje de Ceros del total de elementos de la Matriz Frecuencias

In [11]:
PorcentajeCeros = 100*(X.apply(lambda x: len(np.where(x==0)[0])).sum()/(X.shape[0]*X.shape[1]))
PorcentajeCeros

99.80506833344035

##### Número de Ceros en cada fila

In [12]:
PorecntajeCerosPorArticulo = X.apply(lambda x: 100*len(np.where(x ==0)[0])/len(x), axis=1).sort_values(ascending=False)
PorecntajeCerosPorArticulo.head(10)

Mochila Pocoyo                              99.989554
Cars - DVD Cars                             99.989554
Papelera Spider-Man                         99.984332
Cubo Rubik's                                99.984332
Ultra - Pack 4 pilas 9V Ultra Alcalinas     99.979109
Ultra - Pack 6 Pilas C Ultra Alcalinas      99.979109
Ultra - Pack 6 Pilas D Ultra Alcalinas      99.979109
Ultra - Pack 20 Pilas AA Ultra Alcalinas    99.973886
Colchón Minicuna Espuma 90X40cm             99.973886
Duracell - 4 Pilas Duracell Ultra AA M3     99.973886
dtype: float64

##### Número de Ceros en cada columna

In [13]:
PorecntajeCerosPorPalabra = X.apply(lambda x: 100*len(np.where(x ==0)[0])/len(x), axis=0).sort_values(ascending=True)
PorecntajeCerosPorPalabra.head(10)

años          33.893157
incluidas     79.131678
puede         81.615151
varios        81.793818
set           82.222619
colores       83.437556
cualquier     85.117027
accesorios    85.349294
puedes        86.349830
funciona      86.528497
dtype: float64

##### Ejemplo de los valores diferentes de cero

In [14]:
X.iloc[0,np.where(X.iloc[0]>0)[0]]

acero          1
asalvo         2
años           1
capota         1
cesta          1
chasis         1
cubrepiés      1
dobles         1
elba           2
inferior       1
ligera         1
máxima         1
máximo         1
parte          1
paseo          2
peso           1
posiciones     1
reclinable     1
recomendado    1
respaldo       1
rosa           2
ruedas         1
silla          2
varias         1
ventana        1
Name: Asalvo - Silla de Paseo Elba Rosa, dtype: int64

##### Cálculo término frecuencia inversa de documento (TfidfTransformer())

https://towardsdatascience.com/text-vectorization-term-frequency-inverse-document-frequency-tfidf-5a3f9604da6d

In [15]:
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(MatrizFrecuencias)
tfidf

<5597x19147 sparse matrix of type '<class 'numpy.float64'>'
	with 208900 stored elements in Compressed Sparse Row format>

##### Visualizacion Matriz con términos frecuencia inversa de documento y normalizada

In [16]:
X_norm = pd.DataFrame(data = tfidf.toarray(),index=Listado.values,
                     columns=vectorizer.get_feature_names())
X_norm

C:\Users\rzambrano\Anaconda3\envs\general\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,_ardtaloqko,aaa,aaron,abajo,abalanzarse,abalorio,abalorios,abandonada,abandonadas,abandonar,...,única,únicamente,únicas,único,únicos,úsala,úsela,útil,útiles,überjackal
Asalvo - Silla de Paseo Elba Rosa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zobo - Silla Paseo Moving Light,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maclaren - Silla Paseo Quest Medieval Azul/Plata,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicco - Silla Paseo London - Red Passion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicco - Silla Paseo London - Matrix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Magic Box Huella,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bright Starts - Mordedores Nuby,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bright Stats - Mordedor Libro (varios modelos),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Peppa Pig - Toalla de Playa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Ejemplo de los valores diferentes de cero

In [17]:
X_norm.iloc[0, np.where(X.iloc[0] > 0)[0]]

acero          0.149286
asalvo         0.385395
años           0.039728
capota         0.153743
cesta          0.158001
chasis         0.177994
cubrepiés      0.205904
dobles         0.183749
elba           0.479472
inferior       0.170702
ligera         0.159577
máxima         0.148546
máximo         0.140922
parte          0.100676
paseo          0.246332
peso           0.122149
posiciones     0.126457
reclinable     0.144477
recomendado    0.198532
respaldo       0.145769
rosa           0.228921
ruedas         0.104590
silla          0.232274
varias         0.125800
ventana        0.168391
Name: Asalvo - Silla de Paseo Elba Rosa, dtype: float64

## Sistema Recomendación por Similitud entre items

Para cada objeto se calculan las similitud con el resto de objetos.

<br>
<br>

$$Similitud_{item} (x_1, x_2) = cos (\theta) = \frac {v_{x_1} \cdot v_{x_2}}{||v_{x_1}|| \cdot ||v_{x_2}||} = \frac {\sum_{i=1}^N (v_{x_1} * v_{x_2})}{\sum_{i=1}^N(v_{x_1,i}^2) * \sum_{i=1}^N(v_{x_2,i}^2)}$$

<br>

Para cada objeto se recomiendan los objetos con similitud más elevada.

<br>

##### Cálculo matriz similitudes

In [18]:
tdm = tfidf.transpose()
dtm = tfidf
Simil = dtm.dot(tdm)

##### Validación de que la matriz anterior estaba normalizada (ver diagonal)

In [19]:
Simil.diagonal()

array([1., 1., 1., ..., 1., 1., 1.])

##### Visualizacion Matriz Similitudes

In [20]:
SimilDF = pd.DataFrame(data = Simil.toarray(), index=Listado.values,columns=Listado.values)
SimilDF

,Asalvo - Silla de Paseo Elba Rosa,Zobo - Silla Paseo Moving Light,Maclaren - Silla Paseo Quest Medieval Azul/Plata,Chicco - Silla Paseo London - Red Passion,Chicco - Silla Paseo London - Matrix,Chicco - Silla de Paseo Echo Twin - Garnet,Num Noms - Pack Misterioso (varios modelos),Princesas Disney - Set Cenicienta,PJ Masks - Figura de Gatuno (Catboy) con Luz,You & Me - Pack Baberos y Toalla,...,Mickey Mouse - Set de Cuna 2 Piezas Mickey,Soy Luna - Juego Sabanas 3 Piezas Soy Luna Love,Patrulla Canina - Juego Sábanas Rosa,Patrulla Canina - Juego Sábanas Leaders,Marco Foto y Doble Huella,Magic Box Huella,Bright Starts - Mordedores Nuby,Bright Stats - Mordedor Libro (varios modelos),Peppa Pig - Toalla de Playa,Star Wars - Toalla de Playa
Asalvo - Silla de Paseo Elba Rosa,1.000000,0.201069,0.188081,0.186962,0.189479,0.153906,0.000851,0.001142,0.001692,0.009521,...,0.000000,0.000000,0.034923,0.000884,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Zobo - Silla Paseo Moving Light,0.201069,1.000000,0.221097,0.184993,0.187483,0.136918,0.000000,0.000000,0.000000,0.004439,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006990,0.005435,0.000000,0.000000
Maclaren - Silla Paseo Quest Medieval Azul/Plata,0.188081,0.221097,1.000000,0.174654,0.177005,0.163220,0.000000,0.009771,0.000000,0.000000,...,0.027313,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chicco - Silla Paseo London - Red Passion,0.186962,0.184993,0.174654,1.000000,0.889713,0.402382,0.000549,0.000737,0.001092,0.013661,...,0.000000,0.000000,0.000659,0.000570,0.000000,0.006288,0.000000,0.000000,0.000000,0.000000
Chicco - Silla Paseo London - Matrix,0.189479,0.187483,0.177005,0.889713,1.000000,0.407798,0.000556,0.000747,0.001107,0.013845,...,0.000000,0.000000,0.000668,0.000578,0.000000,0.006372,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Magic Box Huella,0.000000,0.000000,0.000000,0.006288,0.006372,0.005833,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.267295,1.000000,0.000000,0.000000,0.000000,0.000000
Bright Starts - Mordedores Nuby,0.000000,0.006990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008404,...,0.000000,0.005523,0.000000,0.000000,0.000000,0.000000,1.000000,0.163379,0.000000,0.006319
Bright Stats - Mordedor Libro (varios modelos),0.000000,0.005435,0.000000,0.000000,0.000000,0.015967,0.090859,0.000000,0.000000,0.013069,...,0.006893,0.004294,0.000000,0.002504,0.000000,0.000000,0.163379,1.000000,0.008570,0.004913
Peppa Pig - Toalla de Playa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.128471,...,0.021227,0.021784,0.015720,0.013604,0.000000,0.000000,0.000000,0.008570,1.000000,0.528797


##### Ejemplo Recomendación por Similitud entre items

In [28]:
Top = 10
Num_juguete = 59
print('Juguete:',SimilDF.columns[Num_juguete])
RecomendacionItemItem = SimilDF.iloc[(-SimilDF.iloc[:, Num_juguete]).argsort()[1:(Top+1)].values, Num_juguete]
print ('\n Los juguetes más similares son:')
RecomendacionItemItem

Juguete: Feber - Baby Feber Trike Premium

 Los juguetes más similares son:


Feber - Baby Feber Trike Premium Rosa             0.695776
Feber - Triciclo Cars                             0.391638
Feber - Triciclo Baby Plus                        0.383244
Feber - Baby Plus Music Rosa                      0.327349
Feber - Triciclo Princesas Disney                 0.322775
Avigo - Triciclo Evolutivo Aluminio Azul/Verde    0.320658
Avigo - Triciclo Evolutivo Aluminio Plata/Rosa    0.316939
Feber - Triciclo Ferrari                          0.291436
Feber - Quad 82 Feber                             0.278058
Triciclo Y Stroller Compact Azul                  0.250755
Name: Feber - Baby Feber Trike Premium, dtype: float64

## Sistema Recomendación Filtrado Colaborativo Item-Item

Para cada usuario se propagan las valoraciones de cada objeto al resto mediante la métrica de similaridad entre los objetos

$$Score(u,x_i) = \sum\limits_{\substack{j=1 \\ j \neq i, \hspace{1mm} j \hspace{0.5mm} \epsilon \hspace{0.5mm} J}}^n w(rank_u(x_j)) * similitud_{item}(x_j, x_i)$$

Donde:
<ul>
    <li>$J$ es el conjunto de objetos valorados por el usuario $u$</li>
    <br>
    <li>$rank_u(x_j)$ es la valoración del usuario $u$ al objeto $x_j$</li>
    <br>
    <li>$w:[0-10] \to \mathbb{R}$ es una función que asigna a cada valoración un valor real</li>
</ul>

##### Simulación valoracion individual de un usuario

In [25]:
# Simulamos valoraciones del 1 al 10
np.random.seed(26)
ValoracionesSimuladas = np.random.randint(1,11,len(Listado))

# 90 % valoraciones iguales a 0 (NA's)
PorcentajeCeros = 0.9
ValoracionesSimuladas[np.random.choice(len(Listado), 
                                       int(PorcentajeCeros*len(Listado)), replace=False)] = 0.0

# Cálculos previos valoración real
ValoracionesSimuladasMedia = ValoracionesSimuladas[np.where(ValoracionesSimuladas !=0)].mean()


# Valoración normalizada
ValoracionesSimuladas[np.where(ValoracionesSimuladas !=0)] = ValoracionesSimuladas[np.where(ValoracionesSimuladas !=0)] - ValoracionesSimuladasMedia

ValoracionesSimuladas

array([0, 0, 0, ..., 0, 4, 0])

##### Ejemplo Recomendación de 10 juguetes por Filtrado Colaborativo Item-Item

In [26]:
# Pasamos a DataFrame las puntuaciones predichas para cada juguete
RecomendacionFiltradoColaborativoItemItem = pd.DataFrame(data = Simil.dot(ValoracionesSimuladas), index = Listado.values, columns = ['Scores'])

# Solo seleccionamos los juguetes que no han sido valorados por el usuario (Se supone que los valorados ya han sido comprados por el usuario)
RecomendacionFiltradoColaborativoItemItem = RecomendacionFiltradoColaborativoItemItem.iloc[np.where(ValoracionesSimuladas == 0)[0],:]

# Ordenación decreciente
RecomendacionFiltradoColaborativoItemItem = RecomendacionFiltradoColaborativoItemItem.iloc[(-RecomendacionFiltradoColaborativoItemItem.iloc[:, 0]).argsort().values, 0]

# Muestra el top 10 de los juguetes recomendados 
pd.DataFrame(RecomendacionFiltradoColaborativoItemItem.index.values, columns = ['Juguete']).head(10)

,Juguete
0,Peppa Pig - Poncho de Playa
1,Minnie Mouse - Toalla 70x140 cm + Bolsa (vario...
2,Buscando a Dory - Poncho de Playa
3,Minnie - Toalla 70x140 cm (varios modelos)
4,Nerf - Elite Strongarm
5,Mickey Mouse - Toalla 70x140 cm + Bolsa (vario...
6,Disney - Mochila de Playa Buscando a Dory
7,Mickey Mouse - Toalla 70x140 cm (varios modelos)
8,Buscando a Dory - Asiento Elevador Dory (De 15...
9,Peppa Pig - Figuras Peppa Pig y sus amigos (va...


## Sistema Recomendación Filtrado Colaborativo User-User

Permite estimar valoraciones de objetos que hayan valorado otros usuarios que tengan valoraciones en común con el usuario objetivo.

$$R(u_a, x) = \frac{\sum_{u_k \epsilon R_x} (z_{u_k, x} * similitud(u_a, u_k))}{\sum_{u_k \epsilon R_x} |similitud(u_a, u_k)|} * \sigma_a + \mu_a$$

$$similitud(u_a, u_k) = \frac{\sum_{i \epsilon L_{a,k}} (\upsilon_{k,i} - \mu_k)(\upsilon_{a,i} - \mu_a)}{\sqrt{\sum_{i \epsilon L_{a,k}} (\upsilon_{k,i} - \mu_k)^2} \sqrt{\sum_{i \epsilon L_{a,k}} (\upsilon_{a,i} - \mu_a)^2}}$$

$$z_{u_k, x} = \frac{\upsilon_{k, x} - \mu_k}{\sigma_k}$$

Donde:
<ul>
    <li>$u_a$ denota al usuario $a$</li>
    <li>$u_k$ denota al usuario $k$</li>
    <li>$x$ denota el objeto $x$ a valorar</li>
    <li>$R_x$ es el conjunto de usuarios que han valorado el objeto $x$</li>
    <li>$z_{u_k, x}$ es la normalización de la valoración del usuario $k$ sobre el objeto $x$</li>
    <li>$\sigma_a$ denota la desviación típica de las valoraciones del usuario $a$</li>
    <li>$\mu_a$ denota la media aritmética de las valoraciones del usuario $a$</li>
    <li>$\mu_k$ denota la media aritmética de las valoraciones del usuario $k$</li>
    <li>$L_{a,k}$ es el listado de objetos que han valorado tanto el usuario $a$ como el usuario $k$</li>
    <li>$\upsilon_{k,i}$ es la valoración media realizada por el usuario $k$ sobre el objeto $i$</li>
    <li>$\upsilon_{a,i}$ es la valoración media realizada por el usuario $a$ sobre el objeto $i$</li>    
</ul>

##### Lectura csv con valoraciones de 4 usuarios

In [29]:
valoracionesUsuarios = pd.read_csv("DatosJuguetes\ValoracionesUsuarios.csv", delimiter=";", encoding='iso-8859-1')
valoracionesUsuarios = valoracionesUsuarios.iloc[:, [0,2,3,4,5]]
valoracionesUsuarios.head(10)

,Titulo,User1,User2,User3,User4
0,Asalvo - Silla de Paseo Elba Rosa,NaN,NaN,NaN,NaN
1,Zobo - Silla Paseo Moving Light,NaN,NaN,NaN,NaN
2,Maclaren - Silla Paseo Quest Medieval Azul/Plata,NaN,NaN,NaN,NaN
3,Chicco - Silla Paseo London - Red Passion,NaN,NaN,NaN,NaN
4,Chicco - Silla Paseo London - Matrix,NaN,NaN,NaN,NaN
5,Chicco - Silla de Paseo Echo Twin - Garnet,5.0,NaN,7.0,1.0
6,Num Noms - Pack Misterioso (varios modelos),NaN,NaN,NaN,NaN
7,Princesas Disney - Set Cenicienta,NaN,NaN,NaN,NaN
8,PJ Masks - Figura de Gatuno (Catboy) con Luz,9.0,NaN,NaN,2.0
9,You & Me - Pack Baberos y Toalla,NaN,NaN,NaN,NaN


##### Ejemplo valoraciones user1

In [30]:
valoracionesUsuarios.iloc[:,1][np.where(valoracionesUsuarios.iloc[:,1].notnull())[0]]

5     5.0
8     9.0
12    4.0
18    2.0
26    1.0
Name: User1, dtype: float64

##### Cálculo valoraciones normalizadas 

In [32]:
# Cálculos previos valoración real (valoracioes normalizadas)
medias = valoracionesUsuarios.iloc[:,1:5].apply(lambda x: x[np.where(x.notnull())[0]].mean(), axis=0)
desviaciones = valoracionesUsuarios.iloc[:,1:5].apply(lambda x: x[np.where(x.notnull())[0]].std(), axis=0)

# Normalización valoraciones
valoracionesUsuariosNorm = (valoracionesUsuarios.iloc[:,1:valoracionesUsuarios.shape[1]] - medias.values)/desviaciones
valoracionesUsuariosNorm = valoracionesUsuariosNorm.fillna(value=0)
valoracionesUsuariosNorm.head(10)

,User1,User2,User3,User4
0,0.000000,0.0,0.000000,0.000000
1,0.000000,0.0,0.000000,0.000000
2,0.000000,0.0,0.000000,0.000000
3,0.000000,0.0,0.000000,0.000000
4,0.000000,0.0,0.000000,0.000000
5,0.256865,0.0,0.440225,-0.979796
6,0.000000,0.0,0.000000,0.000000
7,0.000000,0.0,0.000000,0.000000
8,1.541187,0.0,0.000000,-0.734847
9,0.000000,0.0,0.000000,0.000000


##### Cálculo Similitud entre usuarios

In [33]:
SimilUsuarios = valoracionesUsuariosNorm.corr()
SimilUsuarios

,User1,User2,User3,User4
User1,1.000000,-0.222837,0.325831,-0.635707
User2,-0.222837,1.000000,-0.406663,0.404145
User3,0.325831,-0.406663,1.000000,-0.176090
User4,-0.635707,0.404145,-0.176090,1.000000


##### Cálculo Scores Recomendación

In [34]:
Numerador = valoracionesUsuariosNorm.dot(SimilUsuarios)
Denominador = (valoracionesUsuariosNorm != 0).dot(abs(SimilUsuarios))
Cociente = (Numerador/Denominador).values
Cociente

array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]])

##### Ejemplo Recomendación juguetes por Filtrado Colaborativo User-User

In [35]:
RecomendacionFiltradoColaborativoUserUser = pd.DataFrame(data = Cociente, index = Listado.values)

# Ejemplo:
# Se selecciona usuario
Usuario = 4
# Se seleccionan aquellos elemento que no estan valorados por el usuario 4 y que tienen valores predichos por Filtrado Colaboratibo User-User
ElemntosValidos = RecomendacionFiltradoColaborativoUserUser.iloc[:, (Usuario - 1)].notnull().values * valoracionesUsuarios.iloc[:, Usuario].isnull().values

# Recomendación
RecomendacionFiltradoColaborativoUserUser = RecomendacionFiltradoColaborativoUserUser.iloc[ElemntosValidos, (Usuario - 1)]
pd.DataFrame(RecomendacionFiltradoColaborativoUserUser.iloc[(-RecomendacionFiltradoColaborativoUserUser).argsort().values].index.values, columns = ['Juguete'])

,Juguete
0,Monster High - Maletín
1,Totally Me - Cuentas de Ensueño 3 en 1


## Sistema Recomendación Reducción Dimensionalidad 


Surprise es una librería específicamente diseñada para construir sistemas de recomendación en Python. Puede considerarse como la versión de scikit-learn para algoritmos de recomendación. `surprise` se ha convertido en una de las librerías para algoritmos de recomendacón más populares en python. Esto se debe a que es extremadamente robusto y fácil de usar.

La gran ventaja es que nos ofrece implementaciones de los algoritmos de filtro colaborativo más populares. Además también nos permitiria incorporar nuestro propio algoritomo dentro de su framework.

<center><img src='https://image.slidesharecdn.com/4772391/95/building-a-recommendation-engine-an-example-of-a-product-recommendation-engine-21-728.jpg' width=600 ></center>

In [ ]:
#!pip install surprise

In [2]:
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise import SVD
import pandas as pd

# Un reader es todavia necesario pero solo parametro rating_scale es requerido
reader = Reader(rating_scale=(1, 5))

df = pd.read_csv("DatosJuguetes/u.data", delimiter="\t", header = None, encoding='iso-8859-1')
df.columns= ['userID', 'itemID', 'rating', 'timestamp']
# Las columnas deben corresponder a user id, item id and ratings (en este orden).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# Metodo seleccionado
modelo=SVD()

In [3]:
df

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [40]:
# Cross validation
cross_validate(modelo, data, cv=5)

{'test_rmse': array([0.93754997, 0.93340006, 0.93749522, 0.9340679 , 0.9459332 ]),
 'test_mae': array([0.73992682, 0.73490633, 0.73599697, 0.7372058 , 0.74767153]),
 'fit_time': (6.341140508651733,
  6.100412368774414,
  6.775050401687622,
  6.095577955245972,
  6.098322868347168),
 'test_time': (0.16567659378051758,
  0.23258185386657715,
  0.15220046043395996,
  0.1715996265411377,
  0.15798497200012207)}

In [41]:
# train and tets
trainset, testset = train_test_split(data, test_size=.25)

# Entrenamiento y predicción
modelo.fit(trainset)
predictions = modelo.test(testset)
predictions

[Prediction(uid=339, iid=649, r_ui=5.0, est=3.9105217360619284, details={'was_impossible': False}),
 Prediction(uid=925, iid=288, r_ui=5.0, est=3.1441408220354994, details={'was_impossible': False}),
 Prediction(uid=52, iid=815, r_ui=4.0, est=3.880757938566138, details={'was_impossible': False}),
 Prediction(uid=707, iid=479, r_ui=3.0, est=4.046332602685752, details={'was_impossible': False}),
 Prediction(uid=169, iid=879, r_ui=5.0, est=3.3315859554182916, details={'was_impossible': False}),
 Prediction(uid=497, iid=588, r_ui=4.0, est=3.829242427627216, details={'was_impossible': False}),
 Prediction(uid=234, iid=87, r_ui=3.0, est=3.423749221163478, details={'was_impossible': False}),
 Prediction(uid=796, iid=849, r_ui=4.0, est=2.913829666935935, details={'was_impossible': False}),
 Prediction(uid=661, iid=294, r_ui=4.0, est=2.9646421920750856, details={'was_impossible': False}),
 Prediction(uid=159, iid=1013, r_ui=4.0, est=2.6362258591870513, details={'was_impossible': False}),
 Predi

In [42]:
# Accuracy RMSE
accuracy.rmse(predictions)

RMSE: 0.9398


0.9398384830332998

## Deep Learning

In [4]:
import numpy as np
import pandas as pd
import os
import warnings

In [5]:
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model
warnings.filterwarnings('ignore')
%matplotlib inline

In [6]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
n_items = df.itemID.nunique()
n_users = df.userID.nunique()

In [11]:
# creating embedding 
movie_input = Input(shape=[1], name="Movie-Input")
movie_embedding = Embedding(n_items+1, 5, name="Movie-Embedding")(movie_input)
movie_vec = Flatten(name="Flatten-Books")(movie_embedding)
# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)
# concatenate features
conc = Concatenate()([movie_vec, user_vec])
# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)
# Create model and compile it
model2 = Model([user_input, movie_input], out)
model2.compile('adam', 'mean_squared_error')

In [18]:
model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Movie-Input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
User-Input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 5)         8415        Movie-Input[0][0]                
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 5)         4720        User-Input[0][0]                 
______________________________________________________________________________________________

In [12]:
history = model2.fit([train.userID, train.itemID], train.rating, epochs=5, verbose=1)

Epoch 1/5
2500/2500 [==============================] - 2s 762us/step - loss: 1.1418
Epoch 2/5
2500/2500 [==============================] - 2s 787us/step - loss: 0.8784
Epoch 3/5
2500/2500 [==============================] - 2s 742us/step - loss: 0.8562
Epoch 4/5
2500/2500 [==============================] - 2s 766us/step - loss: 0.8385
Epoch 5/5
2500/2500 [==============================] - 2s 794us/step - loss: 0.8186


In [14]:
predictions = model2.predict([test.userID, test.itemID])

In [19]:
from sklearn.metrics import mean_squared_error, r2_score

print(mean_squared_error(test.rating,predictions))
print(r2_score(test.rating,predictions))

0.8682607691239022
0.312547456326871


## Más algoritmos en el paquete `surprise`

No vamos a entrar en detalles del resto de algoritmos que contiene el paquete, pero vamos a listarlos para que los conozcaís y valoreis su utilización en el futuro.




### NormalPredictor

NormalPredictor predice los ratings de forma aleatoria entendiendo que la distribución de las puntuaciones sigue una normal. Es uno de los algoritmos más básicos que podemos encontrar.




### BaselineOnly
BaselineOnly nos devolverá el baseline para la combinación de usuario e ítem.




### k-NN algorithms

#### KNNBasic
KNNBasic es un algoritmo básico de filtro colaborativo.



#### KNNWithMeans
KNNWithMeans es un algoritmo de filtro colaborativo, que tienen en cuenta la media de los ratings de cada usuario. 



#### KNNWithZScore
KNNWithZScore es un algoritmo de filtro colaborativo, que tienen en cuenta la normalización z-score de las puntuaciones de cada usuario.



#### KNNBaseline
KNNBaseline es un algoritmo de filtro colaborativo, basado en el baseline. 


### Matrix Factorization-based algorithms


#### SVD y SVDpp

SVD y SVDpp son dos versiones del algoritmo que hemos analizado anteriormente.



#### NMF

NMF  es un algoritmo de filtro colaborativo basado en Non-negative Matrix Factorization. Conceptualmente es muy similar a  SVD.


### Slope One
SlopeOne es una implementación del algoritmo SlopeOne. https://en.wikipedia.org/wiki/Slope_One



### Co-clustering

Algortimo de filtro colaborativo basado en co-clustering.